In [1]:
import time
import math
import requests
import numpy as np
import pandas as pd
from helper import get_authorization_url, get_access_token, get_playlist_ids_from_categories, get_song_ids_from_playlist, get_spotify_songs_metadata

In [2]:
auth_url = get_authorization_url()
print(auth_url)

https://accounts.spotify.com/authorize?client_id=962aeb5559434e8a8ddb2b8c78e6c3d2&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3000&scope=user-read-private+user-read-email


In [3]:
cookie = "AQCg5p-O3JZv3nRM63Ogc8eAAoS-0xkaMkKx1Q6qbAPMvpyKCfxeKbSRW9IWCNED87R5sOzFiHfW8oiIEs5x4PIphfgG5_PkjW3ymkw3c5D0T0PrKSLE65NFiEzRR2cAOBmdSqiSUmDXb71y153jdG_uIXqukzhm4GKVJX3zpRs_UVRH5d4hlwYHMkcZ_XCfrp8-vwGAb6QOEtR4Rw"

In [4]:
access_token = get_access_token(cookie)

KeyError: 'access_token'

In [9]:
url = "https://api.spotify.com/v1/me/playlists"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    playlists = response.json()['items']
else:
    print(f'Error: {response.status_code} - {response.json()}')

Error: 400 - {'error': {'status': 400, 'message': 'Only valid bearer authentication supported'}}


In [ ]:
favorite_songs = []

for playlist in playlists:
  song_url = f"https://api.spotify.com/v1/playlists/{playlist['id']}/tracks"
  headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
  }
  response = requests.get(song_url, headers=headers)

  if response.status_code == 200:
    favorite_songs.extend([song['track']['id'] for song in response.json()['items']])

batched_favorite_songs = [favorite_songs[i:i + 50] for i in range(0, len(favorite_songs), 50)]

In [ ]:
# liked_songs = pd.DataFrame()
# for batch in batched_favorite_songs:
#     liked_songs = get_spotify_songs_metadata(liked_songs, access_token, batch, 1, 1)
# liked_songs.to_parquet('./datasets/cleaned/music-recommendation-system/liked/songs.parquet', engine='pyarrow')

In [ ]:
genres = ['pop', 'mood', 'gaming']

playlist_ids = get_playlist_ids_from_categories(access_token, ['pop'], 'TW', 1)
unheard_song_ids = get_song_ids_from_playlist(access_token, playlist_ids)

In [ ]:
print(playlist_ids)

In [ ]:
batches = math.ceil(len(unheard_song_ids) / 10)
unheard_song_ids_batches = np.array_split(unheard_song_ids, batches)

In [ ]:
unheard_songs = pd.DataFrame()

for batch in unheard_song_ids_batches:
    unheard_songs = get_spotify_songs_metadata(unheard_songs, access_token, batch, 0, np.nan)

In [ ]:
unheard_songs.to_parquet('./datasets/cleaned/music-recommendation-system/unheard/songs.parquet', engine='pyarrow')